# Assignment 4 - Using NLP to play the stock market

In this assignment, we'll use everything we've learned to analyze corporate news and pick stocks. Be aware that in this assignment, we're trying to beat the benchmark of random chance (aka better than 50%).

This assignment will involve building three models:

**1. An RNN based on word inputs**

**2. A CNN based on character inputs**

**3. A neural net architecture that merges the previous two models**

You will apply these models to predicting whether a stock return will be positive or negative in the same day of a news publication.

## Your X - Reuters news data

Reuters is a news outlet that reports on corporations, among many other things. Stored in the `news_reuters.csv` file is news data listed in columns. The corresponding columns are the `ticker`, `name of company`, `date of publication`, `headline`, `first sentence`, and `news category`.

In this assignment it is up to you to decide how to clean this dataset. For instance, many of the first sentences contain a location name showing where the reporting is done. This is largely irrevant information and will probably just make your data noisier. You can also choose to subset on a certain news category, which might enhance your model performance and also limit the size of your data.

## Your Y - Stock information from Yahoo! Finance

Trading data from Yahoo! Finance was collected and then normalized using the [S&P 500](https://en.wikipedia.org/wiki/S%26P_500_Index). This is stored in the `stockReturns.json` file. 

In our dataset, the ticker for the S&P is `^GSPC`. Each ticker is compared the S&P and then judged on whether it is outperforming (positive value) or under-performing (negative value) the S&P. Each value is reported on a daily interval from 2004 to now.

Below is a diagram of the data in the json file. Note there are three types of data: short: 1 day return, mid: 7 day return, long 28 day return.

```
          term (short/mid/long)
         /         |         \
   ticker A   ticker B   ticker C
      /   \      /   \      /   \
  date1 date2 date1 date2 date1 date2
```

You will need to pick a length of time to focus on (day, week, month). You are welcome to train models on each dataset as well.  

Transform the return data such that the outcome will be binary:

```
label[y < 0] = 0
label[y >= 0] = 1
```

Finally, this data needs needs to be joined on the date and ticker - For each date of news publication, we want to join the corresponding corporation's news on its return information. We make the assumption that the day's return will reflect the sentiment of the news, regardless of timing.


# Your models - RNN, CNN, and RNN+CNN

For your RNN model, it needs to be based on word inputs, embedding the word inputs, encoding them with an RNN layer, and finally a decoding step (such as softmax or some other choice).

Your CNN model will be based on characters. For reference on how to do this, look at the CNN class demonstration in the course repository.

Finally you will combine the architecture for both of these models, either [merging](https://github.com/ShadyF/cnn-rnn-classifier) using the [Functional API](https://keras.io/getting-started/functional-api-guide/) or [stacking](http://www.aclweb.org/anthology/S17-2134). See the links for reference.

For each of these models, you will need to:
1. Create a train and test set, retaining the same test set for every model
2. Show the architecture for each model, printing it in your python notebook
2. Report the peformance according to some metric
3. Compare the performance of all of these models in a table (precision and recall)
4. Look at your labeling and print out the underlying data compared to the labels - for each model print out 2-3 examples of a good classification and a bad classification. Make an assertion why your model does well or poorly on those outputs.
5. For each model, calculate the return from the three most probable positive stock returns. Compare it to the actual return. Print this information in a table.

### Good luck!

## Load and Clean Raw Data

In [1]:
# Utility libraries
import os
import pickle
import numpy as np
import pandas as pd
import re
import calendar

# Prepocessing libraries
from sklearn.model_selection import train_test_split
import gensim

from keras.models import Model
from keras.layers import Input, concatenate, Concatenate, TimeDistributed
from keras.layers import Dense, Bidirectional, Dropout, Flatten, multiply
from keras.layers import Conv1D, Conv2D, MaxPool1D, BatchNormalization
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras import regularizers

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import keras.backend as K

def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [3]:
dataPath = '../data'
reutersFile = 'news_reuters.csv'
stockFile = 'stockReturns.json'

rawX = pd.read_csv(os.path.join(dataPath, reutersFile), header=None, 
                   names=['ticker', 'company', 'pub_date', 'headline', 'first_sent', 'category'])
rawY = pd.read_json(os.path.join(dataPath, stockFile))
# rawY = json.load(os.path.join(dataPath, stockFile))

#### Reformat and Merge Data

In [4]:
def reformat_y_data(data, tickerType='mid'):
    """Convert stock data into binary postive/negative"""
    tmp = data[tickerType].apply(pd.Series)
    tmp = tmp.stack().rename('price', inplace=True).reset_index()
    tmp['y'] = np.where(tmp['price'] >= 0, 1, 0)
    tmp.rename(columns={'level_0': 'ticker', 'level_1': 'pub_date'}, inplace=True)
    return tmp

def clean_and_merge_data(X, Y):
    """Filter X to only those tickers with stock data"""
    y_tickers = set(Y['ticker'])
    X = X.loc[X['ticker'].isin(y_tickers)]
    # Make sure data types are the same for merge    
    Y['pub_date'] = Y['pub_date'].astype(rawX['pub_date'].dtype)
    Y['ticker'] = Y['ticker'].astype(rawX['ticker'].dtype)
    return X.merge(Y, on=['ticker', 'pub_date'], how='left')

In [5]:
cleanY = reformat_y_data(rawY, 'short')

merged = clean_and_merge_data(rawX, cleanY)

#### Clean up text columns and tokenize data

In [6]:
def clean_text(sent):
    """Clean up text data by:
    
    1. Replacing double spaces into a single space
    2. Replace U.S. to United States so U won't get deleted with next 
       replacement
    3. Remove all capitalized words at the beginning of the 
       sentence, since those are mostly places (aka NEW YORK)
    4. Remove unnecessary punctuation (hyphens and asterisks)
    5. Remove dates
    """
    monthStrings = list(calendar.month_name)[1:] + list(calendar.month_abbr)[1:]
    monthPattern = '|'.join(monthStrings)
    
    sent = re.sub(r' +', ' ', sent)
    sent = re.sub(r'U.S.', 'United States', sent)
    sent = re.sub(r'^(\W?[A-Z\s\d]+\b-?)', '', sent)
    sent = re.sub(r'^ ?\W ', '', sent)
    sent = re.sub(r'({}) \d+'.format(monthPattern), '', sent)
    
    # replace double spaces one more time after previous cleaning 
    sent = re.sub(r' +', ' ', sent)
    return sent 

In [7]:
def tokenize_sent(col):
    """Tokenize string into a sequence of words"""
    return [text_to_word_sequence(text, lower=False) for text in col]

def filt_to_one(x, random_state=10):
    """Filter dataset so that there is only one observation per day.
    
    If there is more than one record, will use the topStory record
    if one exists.  If one doesn't or there are 2 topStory records
    then it will randomly select one of the observations.
    """
    if x.shape[0] > 1:
        if 'topStory' in x['category'].unique():
            x = x.loc[x['category'] == 'topStory']
        if x.shape[0] > 1:
            x = x.sample(n=1, random_state=random_state)
    return x

In [8]:
# Clean up text
merged['headline'] = merged.headline.apply(clean_text)
merged['first_sent'] = merged.first_sent.apply(clean_text)

# Turn sentences into tokens
merged['headline_token'] = tokenize_sent(merged.headline)
merged['first_sent_token'] = tokenize_sent(merged.first_sent)

# Get one record per company/day
finalData = merged.groupby(by=['ticker', 'pub_date']).apply(filt_to_one)

# Combine Headline and First Sentence into one text 
finalData['final_text'] = finalData['headline_token'] + finalData.first_sent_token

# Remove observations with missing stock price
finalData.dropna(inplace=True)

In [61]:
# split into train and test
train, test = train_test_split(finalData, test_size = .2, random_state=10)

#### Create Lexicon and Transform Data to Integers for Modeling

In [62]:
class lexiconTransformer():
    """Create a lexicon and transform sentences and
       to indexes for use in the model."""
    
    def __init__(self, words_min_freq = 1, unknown_word_token = u'<UNK>',
                 savePath='models', saveName='stock_word_lexicon'):
        self.words_min_freq = words_min_freq
        self.words_lexicon = None
        self.unknown_word_token = unknown_word_token
        self.indx_to_words_dict = None
        self.savePath = savePath
        self.saveName = saveName + '.pkl'
    
    def fit(self, sents):
        """Create lexicon based on sentences"""
        self.make_words_lexicon(sents)        
        self.make_lexicon_reverse()
        self.save_lexicon()
                
    def transform(self, sents):
        sents_indxs = self.tokens_to_idxs(sents, self.words_lexicon)
        return sents_indxs

    def fit_transform(self, sents):
        self.fit(sents)
        return self.transform(sents)
        
    def make_words_lexicon(self, sents_token):
        """Wrapper for words lexicon"""
        self.words_lexicon = self.make_lexicon(sents_token, self.words_min_freq,
                                               self.unknown_word_token)

    def make_lexicon(self, token_seqs, min_freq=1, unknown = u'<UNK>'):
        """Create lexicon from input based on a frequency

            Parameters:
            
            token_seqs
            ----------
               A list of a list of input tokens that will be used to create the lexicon
            
            min_freq
            --------
               Number of times the token needs to be in the corpus to be included in the
               lexicon.  Otherwise, will be replaced with the "unknown" entry
            
            unknown
            -------
               The word in the lexicon that should be used for tokens not existing in lexicon.
               This can be a value that already exists in input list.  For instance, in 
               Named Entity Recognition, a value of "other" or "O" may already be a tag 
               and so having "other" and "unknown" are the same thing!
        """
        # Count how often each word appears in the text.
        token_counts = {}
        for seq in token_seqs:
            for token in seq:
                if token in token_counts:
                    token_counts[token] += 1
                else:
                    token_counts[token] = 1

        # Then, assign each word to a numerical index. 
        # Filter words that occur less than min_freq times.
        lexicon = [token for token, count in token_counts.items() if count >= min_freq]
        
        # Have to delete unknown value from token list so not a gap in lexicon values when
        # turning it into a lexicon (aka, if unknown == OTHER and that is the 7th value, 
        # then 7 won't exist in the lexicon which may cause issues)
        if unknown in lexicon:
            lexicon.remove(unknown)

        # Indices start at 1. 0 is reserved for padding, and 1 is reserved for unknown words.
        lexicon = {token:idx + 2 for idx,token in enumerate(lexicon)}
        
        lexicon[unknown] = 1 # Unknown words are those that occur fewer than min_freq times
        lexicon_size = len(lexicon)
        return lexicon
    
    def save_lexicon(self):
        "Save lexicons by pickling them"
        if not os.path.exists(self.savePath):
            os.makedirs(self.savePath)
        with open(os.path.join(self.savePath, self.saveName), 'wb') as f:
            pickle.dump(self.words_lexicon, f)
                        
    def load_lexicon(self):
        with open(os.path.join(self.savePath, self.saveName), 'rb') as f:
            self.words_lexicon = pickle.load(f)
                    
        self.make_lexicon_reverse()
        
    def make_lexicon_reverse(self):
        self.indx_to_words_dict = self.get_lexicon_lookup(self.words_lexicon)
    
    def get_lexicon_lookup(self, lexicon):
        '''Make a dictionary where the string representation of 
           a lexicon item can be retrieved from its numerical index'''
        lexicon_lookup = {idx: lexicon_item for lexicon_item, idx in lexicon.items()}
        return lexicon_lookup
    
    def tokens_to_idxs(self, token_seqs, lexicon):
        """Transform tokens to numeric indexes or <UNK> if doesn't exist"""
        idx_seqs = [[lexicon[token] if token in lexicon else lexicon['<UNK>'] for 
                                 token in token_seq] for token_seq in token_seqs]
        return idx_seqs

In [63]:
lexicon = lexiconTransformer(words_min_freq=2)

lexicon.fit(train['final_text'])

In [64]:
train['finalText_indx'] = lexicon.transform(train['final_text'])
test['finalText_indx'] = lexicon.transform(test['final_text'])

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [65]:
def get_max_seq_len(sents):
    return max([len(idx_seq) for idx_seq in sents])

def pad_idx_seqs(idx_seqs, max_seq_len):
    # Keras provides a convenient padding function; 
    padded_idxs = pad_sequences(sequences=idx_seqs, maxlen=max_seq_len)
    return padded_idxs

In [66]:
# Get length of longest sequence
max_seq_len = get_max_seq_len(train['finalText_indx'])

#Add one to max length for offsetting sequence by 1
train_padded_words = pad_idx_seqs(train['finalText_indx'], 
                                  max_seq_len + 1) 

test_padded_words = pad_idx_seqs(test['finalText_indx'], 
                                  max_seq_len + 1) 

train_y = to_categorical(train['y'])
test_y = to_categorical(test['y'])

## Model 1: RNN

In [22]:
n_out = 2
nb_epoch = 50

In [23]:
def create_embed_matrix(model, lexicon, embed_size):
    "Create a weight matrix for words"
    vocab_size = len(lexicon)
    embedding_matrix = np.zeros((vocab_size, embed_size))
    n = 0
    word_list = list(lexicon)
    for i in range(vocab_size):
        word = word_list[i]
        if word in model.wv.vocab:
            embedding_vector = model.wv[word]
            if embedding_vector is not None:
                embedding_matrix[n] = embedding_vector[:embed_size]
                n += 1

    return embedding_matrix[:n, :]


In [24]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('../data/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [25]:
word_embed_len = 300
word_embed_matrix = create_embed_matrix(w2v, lexicon.words_lexicon, 
                                   word_embed_len)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [37]:
def create_rnn_model(word_input, embed_matrix, n_RNN_nodes,
                     n_rnn_layers=1, 
                     rnn_bidirectional=True,
                     recurrent_dropout=None, 
                     activity_regularizer = None,
                     recurrent_regularizer = None,
                     dropout=None,
                     batch_normalization=True):
    
    "Wrapper function to dynamically create LSTM layers in a model"
           
    word_embeddings = Embedding(input_dim=embed_matrix.shape[0],
                                output_dim=embed_matrix.shape[1],
                                weights=[embed_matrix], 
                                mask_zero=True, 
                                name='word_embedding_layer')(word_input) 

    # Build RNN Layers
    rnn_layers = []
    for i in range(n_rnn_layers):
        if i == (n_rnn_layers - 1):
            return_seq = False
        else:
            return_seq = True
        
        rnn_layer = LSTM(n_RNN_nodes, return_sequences=return_seq,
                         recurrent_dropout=recurrent_dropout,
                         activity_regularizer = activity_regularizer,
                         recurrent_regularizer = recurrent_regularizer,
                         dropout=dropout, 
                         name='hidden_layer{}'.format(i))
        
        if rnn_bidirectional:
            rnn_layer = Bidirectional(rnn_layer)
        rnn_layers.append(rnn_layer)
    
    # Call RNN layers
    for layer_num, layer in enumerate(rnn_layers):
        
        if layer_num == 0:
            rnn_layer = layer(word_embeddings)
        else:
            rnn_layer = layer(rnn_layer)
        
        # Apply batch normalization if requested
        if batch_normalization:
            # If not last layer, need to do TimeDistributed
            if layer_num != n_rnn_layers - 1:
                batch_norm = TimeDistributed(BatchNormalization())
            else:
                batch_norm = BatchNormalization()
                                             
            rnn_layer = batch_norm(rnn_layer)
        
    return rnn_layer

In [38]:
def create_dense_layers(input_layer, n_dense_layers, n_dense_nodes, 
                        dropout=None, batch_normalization=True,
                        activity_regularizer=None,
                        dense_activation='elu'):
    
        # Build RNN Layers
    dense_layers = []
    for i in range(n_dense_layers):
       
        dense_layers.append(Dense(n_dense_nodes, 
                                  activation=dense_activation, 
                                  activity_regularizer=activity_regularizer,
                                  name='dense_layer{}'.format(i)))   
    # Call RNN layers
    for layer_num, layer in enumerate(dense_layers):
        
        if layer_num == 0:
            dense_layer = layer(input_layer)
        else:
            dense_layer = layer(dense_layer)
        
        # Apply Dropout and batch normalization if requested
        if dropout:
            dense_layer = Dropout(dropout)(dense_layer)
        if batch_normalization:
            dense_layer = BatchNormalization()(dense_layer)
        
    return dense_layer

In [39]:
def create_rnn_standalone_model(seq_input_len, embed_matrix, n_RNN_nodes, 
                                n_dense_nodes, n_out=2, 
                                n_dense_layers=1,
                                n_rnn_layers=1, 
                                rnn_bidirectional=True,
                                recurrent_dropout=None, 
                                activity_regularizer = None,
                                recurrent_regularizer = None,
                                dropout=None,
                                batch_normalization=True,
                                dense_activation='elu'):
    
    word_input = Input(shape=(seq_input_len,), name='word_input_layer')
    
    rnn_model = create_rnn_model(word_input, embed_matrix, n_RNN_nodes, 
                                 n_rnn_layers, rnn_bidirectional, 
                                 recurrent_dropout, activity_regularizer, 
                                 recurrent_regularizer, dropout, batch_normalization)

    dense_layers = create_dense_layers(rnn_model,
                                       n_dense_layers, 
                                       n_dense_nodes, 
                                       dropout, 
                                       batch_normalization,
                                       activity_regularizer,
                                       dense_activation)
    
    output_layer = Dense(n_out, activation='softmax',
                         name='output_layer')(dense_layers)

    model = Model(inputs=[word_input], outputs=output_layer)
    model.compile(loss='categorical_crossentropy', optimizer="adam", 
                  metrics=['accuracy', recall, precision])

    return model 

In [40]:
rnn_model = create_rnn_standalone_model(seq_input_len=train_padded_words.shape[-1],
                                        embed_matrix=word_embed_matrix,
                                        n_RNN_nodes=250,
                                        n_dense_nodes=200, 
                                        n_out=2, 
                                        n_dense_layers=1,
                                        n_rnn_layers=1, 
                                        rnn_bidirectional=True,
                                        recurrent_dropout=.4, 
                                        activity_regularizer = None,
                                        recurrent_regularizer = None,
                                        dropout=.2,
                                        batch_normalization=True,
                                        dense_activation='elu')

In [41]:
def train_and_test_model(model, x_train, y_train, x_test, y_test, 
                         modelSaveName, modelSavePath='models',
                         batch_size=128, epochs=3, validation_split=.1):
    """Train model, save weights, and predict data"""
    
    print(model.summary())
    
    filepath = os.path.join(modelSavePath, modelSaveName + '.hdf5')
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1)
    callbacks_list = [checkpoint]
    model.fit(x=x_train, y=y_train, batch_size=batch_size, 
              epochs=epochs, validation_split=validation_split, 
              callbacks=callbacks_list)
    
    score, acc, rec, prec = model.evaluate(x_test, y_test, batch_size=batch_size)
    return (model, acc, rec, prec)    

In [42]:
rnn_res = train_and_test_model(rnn_model, train_padded_words, train_y, 
                               test_padded_words, test_y, 'rnn_model',
                               epochs=nb_epoch)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_input_layer (InputLayer (None, 102)               0         
_________________________________________________________________
word_embedding_layer (Embedd (None, 102, 300)          3534000   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 500)               1102000   
_________________________________________________________________
batch_normalization_5 (Batch (None, 500)               2000      
_________________________________________________________________
dense_layer0 (Dense)         (None, 200)               100200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 200)               800       
__________

Epoch 26/50
8200/8200 [==============================] - 25s 3ms/step - loss: 0.0716 - acc: 0.9673 - recall: 0.9673 - precision: 0.9673 - val_loss: 2.5071 - val_acc: 0.5208 - val_recall: 0.5208 - val_precision: 0.5208

Epoch 00026: saving model to models/rnn_model.hdf5
Epoch 27/50
8200/8200 [==============================] - 25s 3ms/step - loss: 0.0798 - acc: 0.9622 - recall: 0.9622 - precision: 0.9622 - val_loss: 2.2547 - val_acc: 0.5351 - val_recall: 0.5351 - val_precision: 0.5351

Epoch 00027: saving model to models/rnn_model.hdf5
Epoch 28/50
8200/8200 [==============================] - 25s 3ms/step - loss: 0.0679 - acc: 0.9661 - recall: 0.9661 - precision: 0.9661 - val_loss: 2.6485 - val_acc: 0.5362 - val_recall: 0.5362 - val_precision: 0.5362

Epoch 00028: saving model to models/rnn_model.hdf5
Epoch 29/50
8200/8200 [==============================] - 25s 3ms/step - loss: 0.0654 - acc: 0.9652 - recall: 0.9652 - precision: 0.9652 - val_loss: 2.7770 - val_acc: 0.5285 - val_recall: 0.5

KeyboardInterrupt: 

In [ ]:
rnn_res

## Model 2: CNN

In [55]:
def vectorize_sentences(data, lexicon, maxlen=200):
    X = []
    for sentences in data:
        x = [lexicon[token] if token in lexicon else lexicon['<UNK>'] for 
                                 token in sentences]
        x2 = np.eye(len(char_indices) + 1)[x]
        X.append(x2)
    return (pad_sequences(X, maxlen=maxlen))

In [75]:
def validate_args(arg, n_conv_layers, warning=False):
    """Makes sure the convolution parameters equal the size
       of the convoluation model"""
    if len(arg) < n_conv_layers and len(arg) != 1:
        raise ValueError("""One of the values passed to the following arguments 
                            has an incorrect value, either needs to be a value of 1 or a
                            list/tuple of length >= n_conv_layers: {}

                            nb_filter, filter_kernels, or pool_size""".format(n_conv_layers))

    if len(arg) == 1:
        arg = arg * n_conv_layers

    if len(arg) > n_conv_layers and warning:
        print("""Length of argument {} > n_conv_layers. Will only use part of the
               argument amount""".format(len(arg)))
        
    return arg


def create_cnn_model(input_layer, char_maxlen, vocab_size,
                     n_conv_layers=1,
                     nb_filter=None,
                     filter_kernels=None,
                     pool_size=None, 
                     dropout=None,
                     activity_regularizer=None,
                     batch_normalization=True,
                     cnn_padding='same',
                     cnn_activation='elu'):
    """Function to dynamically create convolution and maxpool
       layers."""
    
    # Make sure arguments match number of layers
    nb_filter = validate_args(nb_filter, n_conv_layers)
    filter_kernels = validate_args(filter_kernels, n_conv_layers)
    pool_size = validate_args(pool_size, n_conv_layers)
    

    # Create convolution layers
    for i in range(n_conv_layers):
        if i == 0:
            conv_layer = Conv1D(nb_filter[i], kernel_size=filter_kernels[i],
                                padding=cnn_padding, 
                                activation=cnn_activation,
                                activity_regularizer = activity_regularizer,
                                input_shape=(char_maxlen, vocab_size),
                                name='Conv_layer{}'.format(i))(input_layer)
        else:
            conv_layer = Conv1D(nb_filter[i], kernel_size=filter_kernels[i],
                                padding=cnn_padding, 
                                activation=cnn_activation,
                                activity_regularizer = activity_regularizer,
                                name='Conv_layer{}'.format(i))(conv_layer)
            
        if batch_normalization:
            conv_layer = BatchNormalization()(conv_layer)
            
        if dropout:
            conv_layer = Dropout(dropout)(conv_layer)
        
        conv_layer = MaxPool1D(pool_size=pool_size[i],
                               name='MaxPool{}'.format(i))(conv_layer)
    
    # Flatten convolution layer to work with Dense layers
    return Flatten()(conv_layer)



In [76]:
def create_cnn_standalone_model(char_maxlen, vocab_size,
                                n_dense_nodes,
                                n_dense_layers=1, 
                                n_conv_layers=1,
                                nb_filter=None,
                                filter_kernels=None,
                                pool_size=None, 
                                dropout=None,
                                activity_regularizer=None,
                                batch_normalization=True,
                                cnn_padding='same',
                                cnn_activation='elu',
                                dense_activation='elu'):
    
        char_input = Input(shape=(char_maxlen, vocab_size), name='char_input_layer')
        
        cnn_model = create_cnn_model(char_input, char_maxlen, vocab_size,
                                     n_conv_layers, nb_filter, filter_kernels,
                                     pool_size, dropout, activity_regularizer,
                                     batch_normalization, cnn_padding, cnn_activation)
        
        dense_layers = create_dense_layers(cnn_model, n_dense_layers,
                                           n_dense_nodes, dropout,
                                           batch_normalization,
                                           activity_regularizer,
                                           dense_activation)

        output_layer = Dense(n_out, activation='softmax',
                             name='output_layer')(dense_layers)

        model = Model(inputs=[char_input], outputs=output_layer)
        model.compile(loss='categorical_crossentropy', optimizer="adam", 
                      metrics=['accuracy', recall, precision])

        return model 

In [77]:
char_maxlen = 1200 
nb_filter = [75]
dense_outputs = 628
filter_kernels = [7, 3, 3, 3]
pool_size = [5]

In [78]:
# Turn all tokens into one string and then all obs 
# into one overall string
trainTokensAsString = train.final_text.apply(lambda x: ' '.join(x))
testTokensAsString = test.final_text.apply(lambda x: ' '.join(x))
oneTxt = ' '.join(trainTokensAsString)

# Get info about characters
chars = set(oneTxt)
vocab_size = len(chars) + 1
print('total chars:', vocab_size)
char_indices = dict((c, i + 2) for i, c in enumerate(chars))
indices_char = dict((i + 2, c) for i, c in enumerate(chars))

char_indices['<UNK>'] = 1
indices_char[1] = '<UNK>'

total chars: 86


In [79]:
trainCharData = vectorize_sentences(trainTokensAsString, char_indices, char_maxlen)
testCharData = vectorize_sentences(testTokensAsString, char_indices, char_maxlen)

In [80]:
cnn_model = create_cnn_standalone_model(char_maxlen=char_maxlen,
                                        vocab_size=vocab_size,
                                        n_dense_nodes=dense_outputs,
                                        n_dense_layers=1, 
                                        n_conv_layers=4,
                                        nb_filter=nb_filter,
                                        filter_kernels=filter_kernels,
                                        pool_size=pool_size, 
                                        dropout=.2,
                                        activity_regularizer=None,
                                        batch_normalization=True,
                                        cnn_padding='same',
                                        cnn_activation='elu',
                                        dense_activation='elu')


Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [82]:
cnn_res = train_and_test_model(cnn_model, trainCharData[:, :, 1:],
                               train_y, 
                               testCharData[:, :, 1:], 
                               test_y, 
                               'cnn_model',
                               epochs=nb_epoch)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
char_input_layer (InputLayer (None, 1200, 86)          0         
_________________________________________________________________
Conv_layer0 (Conv1D)         (None, 1200, 75)          45225     
_________________________________________________________________
batch_normalization_7 (Batch (None, 1200, 75)          300       
_________________________________________________________________
dropout_3 (Dropout)          (None, 1200, 75)          0         
_________________________________________________________________
MaxPool0 (MaxPooling1D)      (None, 240, 75)           0         
_________________________________________________________________
Conv_layer1 (Conv1D)         (None, 240, 75)           16950     
_________________________________________________________________
batch_normalization_8 (Batch (None, 240, 75)           300       
__________

8200/8200 [==============================] - 10s 1ms/step - loss: 0.7073 - acc: 0.4982 - recall: 0.4982 - precision: 0.4982 - val_loss: 0.6992 - val_acc: 0.5055 - val_recall: 0.5055 - val_precision: 0.5055

Epoch 00019: saving model to models/cnn_model.hdf5
Epoch 20/50
8200/8200 [==============================] - 10s 1ms/step - loss: 0.7056 - acc: 0.5017 - recall: 0.5017 - precision: 0.5017 - val_loss: 0.7408 - val_acc: 0.5066 - val_recall: 0.5066 - val_precision: 0.5066

Epoch 00020: saving model to models/cnn_model.hdf5
Epoch 21/50
8200/8200 [==============================] - 10s 1ms/step - loss: 0.7102 - acc: 0.4966 - recall: 0.4966 - precision: 0.4966 - val_loss: 0.7403 - val_acc: 0.5077 - val_recall: 0.5077 - val_precision: 0.5077

Epoch 00021: saving model to models/cnn_model.hdf5
Epoch 22/50
8200/8200 [==============================] - 10s 1ms/step - loss: 0.7049 - acc: 0.5027 - recall: 0.5027 - precision: 0.5027 - val_loss: 0.6943 - val_acc: 0.5044 - val_recall: 0.5044 - val_pr

8200/8200 [==============================] - 10s 1ms/step - loss: 0.6860 - acc: 0.5083 - recall: 0.5083 - precision: 0.5083 - val_loss: 0.7236 - val_acc: 0.4923 - val_recall: 0.4923 - val_precision: 0.4923

Epoch 00049: saving model to models/cnn_model.hdf5
Epoch 50/50
8200/8200 [==============================] - 10s 1ms/step - loss: 0.6863 - acc: 0.5152 - recall: 0.5152 - precision: 0.5152 - val_loss: 0.7041 - val_acc: 0.4978 - val_recall: 0.4978 - val_precision: 0.4978

Epoch 00050: saving model to models/cnn_model.hdf5
2278/2278 [==============================] - 2s 714us/step


In [ ]:
cnn_res

## Model 3: RNN+CNN

In [89]:
def create_cnn_rnn_model(rnn_input_len, char_maxlen, vocab_size,
                         embed_matrix, n_RNN_nodes,
                         n_rnn_layers=1, 
                         n_conv_layers=1,
                         n_dense_layers=1,
                         rnn_bidirectional=True,
                         recurrent_dropout=None, 
                         activity_regularizer=None,
                         recurrent_regularizer=None, 
                         dropout=None, 
                         batch_normalization=True,                         
                         nb_filter=[100], 
                         filter_kernels = [4],
                         pool_size=[3], 
                         cnn_padding='same', 
                         cnn_activation='elu',
                         n_dense_nodes=100,
                         dense_activation='elu',
                         n_out=2):
    
    word_input = Input(shape=(rnn_input_len,), name='word_input_layer')
    char_input = Input(shape=(char_maxlen, vocab_size), name='char_input_layer')
        
    rnn_model = create_rnn_model(word_input, embed_matrix, n_RNN_nodes, 
                                 n_rnn_layers, rnn_bidirectional, 
                                 recurrent_dropout, activity_regularizer, 
                                 recurrent_regularizer, dropout, batch_normalization)    

    cnn_model = create_cnn_model(char_input, char_maxlen, vocab_size,
                                 n_conv_layers, nb_filter, filter_kernels,
                                 pool_size, dropout, activity_regularizer,
                                 batch_normalization, cnn_padding, cnn_activation)

    merged_layer = Concatenate()([rnn_model, cnn_model])
#     merged_layer = multiply([rnn_model, cnn_model])
    
    dense_layers = create_dense_layers(merged_layer,
                                       n_dense_layers, 
                                       n_dense_nodes, 
                                       dropout, 
                                       batch_normalization,
                                       activity_regularizer,
                                       dense_activation)
    
    output_layer = Dense(n_out, activation='softmax',
                         name='output_layer')(dense_layers)
    
    model = Model(inputs=[word_input, char_input], outputs=[output_layer])

    model.compile(loss='categorical_crossentropy', optimizer="adam", 
                  metrics=['accuracy', recall, precision])    

    return model 

In [90]:
cnn_rnn_model = create_cnn_rnn_model(rnn_input_len=train_padded_words.shape[-1],
                                     char_maxlen=char_maxlen, 
                                     vocab_size=vocab_size,
                                     embed_matrix=word_embed_matrix, 
                                     n_RNN_nodes=550,
                                     n_rnn_layers=1, 
                                     n_conv_layers=4,
                                     n_dense_layers=1,
                                     rnn_bidirectional=True,
                                     recurrent_dropout=.4, 
                                     activity_regularizer=None,
                                     recurrent_regularizer=None, 
                                     dropout=.2, 
                                     batch_normalization=True,                         
                                     nb_filter=nb_filter, 
                                     filter_kernels = filter_kernels,
                                     pool_size=pool_size, 
                                     cnn_padding='same', 
                                     cnn_activation='elu',
                                     n_dense_nodes=400,
                                     dense_activation='elu',
                                     n_out=2)                             

In [91]:
cnn_rnn_res = train_and_test_model(cnn_rnn_model, 
                               [train_padded_words, trainCharData[:, :, 1:]],
                               train_y, 
                               [test_padded_words, testCharData[:, :, 1:]],
                               test_y, 
                               'cnn_rnn_model',
                               epochs=nb_epoch)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input_layer (InputLayer)   (None, 1200, 86)     0                                            
__________________________________________________________________________________________________
Conv_layer0 (Conv1D)            (None, 1200, 75)     45225       char_input_layer[0][0]           
__________________________________________________________________________________________________
batch_normalization_23 (BatchNo (None, 1200, 75)     300         Conv_layer0[0][0]                
__________________________________________________________________________________________________
dropout_16 (Dropout)            (None, 1200, 75)     0           batch_normalization_23[0][0]     
__________________________________________________________________________________________________
MaxPool0 (

8200/8200 [==============================] - 37s 5ms/step - loss: 0.2820 - acc: 0.8771 - recall: 0.8771 - precision: 0.8771 - val_loss: 1.1075 - val_acc: 0.5230 - val_recall: 0.5230 - val_precision: 0.5230

Epoch 00009: saving model to models/cnn_rnn_model.hdf5
Epoch 10/50
8200/8200 [==============================] - 38s 5ms/step - loss: 0.2465 - acc: 0.9009 - recall: 0.9009 - precision: 0.9009 - val_loss: 1.3116 - val_acc: 0.5252 - val_recall: 0.5252 - val_precision: 0.5252

Epoch 00010: saving model to models/cnn_rnn_model.hdf5
Epoch 11/50
8200/8200 [==============================] - 38s 5ms/step - loss: 0.2173 - acc: 0.9099 - recall: 0.9099 - precision: 0.9099 - val_loss: 1.5557 - val_acc: 0.5230 - val_recall: 0.5230 - val_precision: 0.5230

Epoch 00011: saving model to models/cnn_rnn_model.hdf5
Epoch 12/50
8200/8200 [==============================] - 38s 5ms/step - loss: 0.2004 - acc: 0.9187 - recall: 0.9187 - precision: 0.9187 - val_loss: 1.2914 - val_acc: 0.5143 - val_recall: 0.5

Epoch 39/50
8200/8200 [==============================] - 38s 5ms/step - loss: 0.0517 - acc: 0.9728 - recall: 0.9728 - precision: 0.9728 - val_loss: 2.9996 - val_acc: 0.5230 - val_recall: 0.5230 - val_precision: 0.5230

Epoch 00039: saving model to models/cnn_rnn_model.hdf5
Epoch 40/50
8200/8200 [==============================] - 38s 5ms/step - loss: 0.0608 - acc: 0.9707 - recall: 0.9707 - precision: 0.9707 - val_loss: 3.0503 - val_acc: 0.5230 - val_recall: 0.5230 - val_precision: 0.5230

Epoch 00040: saving model to models/cnn_rnn_model.hdf5
Epoch 41/50
8200/8200 [==============================] - 38s 5ms/step - loss: 0.0783 - acc: 0.9648 - recall: 0.9648 - precision: 0.9648 - val_loss: 2.4229 - val_acc: 0.5186 - val_recall: 0.5186 - val_precision: 0.5186

Epoch 00041: saving model to models/cnn_rnn_model.hdf5
Epoch 42/50
8200/8200 [==============================] - 38s 5ms/step - loss: 0.0669 - acc: 0.9668 - recall: 0.9668 - precision: 0.9668 - val_loss: 2.9039 - val_acc: 0.5241 - val

In [34]:
cnn_rnn_res

(<keras.engine.training.Model at 0x7fbb35ff39e8>,
 0.4991220369791126,
 0.4991220369791126,
 0.4991220369791126)

1. Create a train and test set, retaining the same test set for every model
2. Show the architecture for each model, printing it in your python notebook
2. Report the peformance according to some metric
3. Compare the performance of all of these models in a table (precision and recall)
4. Look at your labeling and print out the underlying data compared to the labels - for each model print out 2-3 examples of a good classification and a bad classification. Make an assertion why your model does well or poorly on those outputs.
5. For each model, calculate the return from the three most probable positive stock returns. Compare it to the actual return. Print this information in a table.

#### Put results in a table

In [120]:
def build_models(rnn_input_len,
                 char_maxlen, 
                 vocab_size,
                 embed_matrix, 
                 n_RNN_nodes,
                 n_dense_nodes,
                 n_rnn_layers, 
                 n_conv_layers,
                 n_dense_layers,
                 rnn_bidirectional,
                 recurrent_dropout, 
                 activity_regularizer,
                 recurrent_regularizer, 
                 dropout, 
                 batch_normalization,                         
                 nb_filter,
                 filter_kernels,
                 pool_size, 
                 cnn_padding, 
                 cnn_activation,
                 dense_activation,
                 n_out=2):
    
    cnn_model = create_cnn_standalone_model(char_maxlen=char_maxlen,
                                            vocab_size=vocab_size,
                                            n_dense_nodes=n_dense_nodes,
                                            n_dense_layers=n_dense_layers, 
                                            n_conv_layers=n_conv_layers,
                                            nb_filter=nb_filter,
                                            filter_kernels=filter_kernels,
                                            pool_size=pool_size, 
                                            dropout=dropout,
                                            activity_regularizer=activity_regularizer,
                                            batch_normalization=batch_normalization,
                                            cnn_padding=cnn_padding,
                                            cnn_activation=cnn_activation,
                                            dense_activation=dense_activation)

    rnn_model = create_rnn_standalone_model(seq_input_len=rnn_input_len,
                                            embed_matrix=word_embed_matrix,
                                            n_RNN_nodes=n_RNN_nodes,
                                            n_dense_nodes=n_dense_nodes, 
                                            n_out=n_out, 
                                            n_dense_layers=n_dense_layers,
                                            n_rnn_layers=n_rnn_layers, 
                                            rnn_bidirectional=rnn_bidirectional,
                                            recurrent_dropout=recurrent_dropout, 
                                            activity_regularizer=activity_regularizer,
                                            recurrent_regularizer=recurrent_regularizer,
                                            dropout=dropout,
                                            batch_normalization=batch_normalization,
                                            dense_activation=dense_activation)

    cnn_rnn_model = create_cnn_rnn_model(rnn_input_len=rnn_input_len,
                                         char_maxlen=char_maxlen, 
                                         vocab_size=vocab_size,
                                         embed_matrix=word_embed_matrix, 
                                         n_RNN_nodes=n_RNN_nodes,
                                         n_rnn_layers=n_rnn_layers, 
                                         n_conv_layers=n_conv_layers,
                                         n_dense_layers=n_dense_layers,
                                         rnn_bidirectional=rnn_bidirectional,
                                         recurrent_dropout=recurrent_dropout, 
                                         activity_regularizer=activity_regularizer,
                                         recurrent_regularizer=recurrent_regularizer, 
                                         dropout=dropout, 
                                         batch_normalization=batch_normalization,                         
                                         nb_filter=nb_filter, 
                                         filter_kernels = filter_kernels,
                                         pool_size=pool_size, 
                                         cnn_padding=cnn_padding, 
                                         cnn_activation=cnn_activation,
                                         n_dense_nodes=n_dense_nodes,
                                         dense_activation=dense_activation,
                                         n_out=n_out)
    
    return {'cnn_model': cnn_model,
            'rnn_model': rnn_model,
            'cnn_rnn_model': cnn_rnn_model,
            'rnn_input_len': rnn_input_len,
            'char_maxlen': char_maxlen, 
            'n_RNN_nodes': n_RNN_nodes,
            'n_rnn_layers': n_rnn_layers, 
            'n_conv_layers': n_conv_layers,
            'n_dense_layers': n_dense_layers,
            'rnn_bidirectional': rnn_bidirectional,
            'recurrent_dropout': recurrent_dropout, 
            'activity_regularizer': activity_regularizer,
            'recurrent_regularizer':  recurrent_regularizer, 
            'dropout': dropout, 
            'batch_normalization': batch_normalization,                         
            'nb_filter': nb_filter, 
            'pool_size': pool_size, 
            'cnn_padding': cnn_padding, 
            'cnn_activation': cnn_activation,
            'n_dense_nodes': n_dense_nodes,
            'dense_activation': dense_activation
            }

In [112]:
hyperparameters = {'n_RNN_nodes': [210, 340, 550],
                  'n_dense_nodes': [200, 400, 600],
                  'n_rnn_layers': [1, 2, 3], 
                  'n_conv_layers': [2, 3, 4],
                  'n_dense_layers': [1, 2],
                  'rnn_bidirectional': [True],
                  'recurrent_dropout': [None, .4],
                  'dropout': [None, .4],
                  'activity_regularizer': [None, regularizers.l2()],
                  'recurrent_regularizer': [None, regularizers.l2()],
                  'batch_normalization': [True, False],
                  'nb_filter':  [[25], [75], [125]], 
                  'pool_size': [[3], [4], [5]], 
                  'cnn_padding': ['same'], 
                  'cnn_activation': ['elu'],
                  'dense_activation': ['elu'],
                  'n_out': [2]}


totalIter = 0
for key, val in hyperparameters.items():
    totalIter += len(val) - 1
    
def hyperparameter_search(hyperparameters):

    n_out=2
    nb_epoch = 30
    char_maxlen = 1200 
    vocab_size=vocab_size
    filter_kernels = [7, 5, 3, 3]
    rnn_input_len=train_padded_words.shape[-1]
    
    currentIterDict = {}
    for key in hyperparameters.keys():
        currentIterDict[key] = hyperparameters[key][0]
        pass
        
        
    models = build_models(rnn_input_len=rnn_input_len,
                          char_maxlen=char_maxlen, 
                          vocab_size=vocab_size,
                          embed_matrix=word_embed_matrix, 
                          filter_kernels=filter_kernels,
                          n_RNN_nodes=currentIterDict['n_RNN_nodes'],
                          n_rnn_layers=currentIterDict['n_rnn_layers'],
                          n_conv_layers=currentIterDict['n_conv_layers'],
                          n_dense_layers=currentIterDict['n_dense_layers'],
                          recurrent_dropout=currentIterDict['recurrent_dropout'],
                          activity_regularizer=currentIterDict['activity_regularizer'],
                          recurrent_regularizer=currentIterDict['recurrent_regularizer'],
                          dropout=currentIterDict['dropout'],
                          batch_normalization=currentIterDict['batch_normalization'],
                          nb_filter=currentIterDict['nb_filter'],
                          pool_size=currentIterDict['pool_size'],
                          cnn_padding=currentIterDict['cnn_padding'],
                          cnn_activation=currentIterDict['cnn_activation'],
                          n_dense_nodes=currentIterDict['n_dense_nodes'],
                          dense_activation=currentIterDict['dense_activation']
                          rnn_bidirectional=currentIterDict['rnn_bidirectional'],
                          n_out=n_out)
    
    pd.DataFrame.from_records([cnn_res[1:4], rnn_res[1:4], cnn_rnn_res[1:4]], 
                          columns=['accuracy', 'recall', 'precision'], 
                         index=['cnn_mod', 'rnn_mod', 'cnn_rnn_mod'])

In [35]:
pd.DataFrame.from_records([cnn_res[1:4], rnn_res[1:4], cnn_rnn_res[1:4]], 
                          columns=['accuracy', 'recall', 'precision'], 
                         index=['cnn_mod', 'rnn_mod', 'cnn_rnn_mod'])

,accuracy,recall,precision
cnn_mod,0.494293,0.494293,0.494293
rnn_mod,0.505707,0.505707,0.505707
cnn_rnn_mod,0.499122,0.499122,0.499122


In [36]:
def print_classifications(classifications, classType, test_y, test_text):
    texts = [' '.join(sent) for sent in test_text[classifications.index]]
    stock_movements = np.where(test_y[classifications.index], 'positive', 'negative')
    
    print('Examples of {} predictions:\n'.format(classType))
    for i in range(len(texts)):
        print('Stock movement was {}'.format(stock_movements[i]))
        print('News info:\n{}'.format(texts[i]))
        print('')

In [37]:
def predict_and_print_samples(model, modelName, test_x, test_y=test['y'], test_text = test['final_text']):
    """"Print out predictions of the model"""
    print('Stats for {} model'.format(modelName))
    
    res = model.predict(test_x)
    class_res = np.apply_along_axis(np.argmax, axis=1, arr=res)

    comparisons = class_res == test_y
    good_class = comparisons.loc[comparisons == True].sample(n=3)
    bad_class = comparisons.loc[comparisons == False].sample(n=3)

    print_classifications(good_class, 'correct', test_y, test_text)
    print_classifications(bad_class, 'INcorrect', test_y, test_text)

    
    top3MostProbPosArg = np.argsort(res[:, 1])[-3:]
    top3Y = test_y.iloc[top3MostProbPosArg]
    top3Probs = pd.Series(res[top3MostProbPosArg, 1], index=top3Y.index)
    top3Data = pd.concat([top3Y, top3Probs], axis=1)
    top3Data.columns = ['Actual', 'PositiveProb']
    print('')
    print('Top 3 Most Positive Probability:')
    print(top3Data)


In [38]:
predict_and_print_samples(rnn_res[0], 'RNN', test_padded_words)

predict_and_print_samples(cnn_res[0], 'CNN', testCharData[:, :, 1:])

predict_and_print_samples(cnn_rnn_res[0], 'CNN_RNN', [test_padded_words, testCharData[:, :, 1:]])

Stats for RNN model
Examples of correct predictions:

Stock movement was positive
News info:
Taiwan's UMC posts 2015 net profit Further company coverage 1 32 7890 Taiwan dollars Reporting by Hong Kong newsroom

Stock movement was positive
News info:
Exxon carries out major evacuation from Iraq oil official

Stock movement was positive
News info:
Lone Star buys 900 mln stg Lloyds' debt portfolio United States private equity firm Lone Star said it had bought Lloyds Banking Group's Project Royal portfolio of debt worth more than 900 million pounds 1 4 billion at a price that sources said was a discount of up to 40 percent

Examples of INcorrect predictions:

Stock movement was negative
News info:
United States FDA approves Amgen leukemia drug ahead of schedule United States health regulators on Wednesday approved an Amgen Inc drug that helps the immune system fight a rare type of leukemia more than five months ahead of the expected decision date

Stock movement was negative
News info:
UK 